<center><font size="10"> 🔥Sentiment Analysis🔥 </font></center>

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [6]:
(X_train, y_train), (X_test, Y_test) = keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 2s 0us/step


In [7]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1

In [8]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

### X_train consists of a list of reviews, each of which is represented as a NumPy array of integers, where each integer represents a word. All punctuation was removed, and then words were converted to lowercase, split by spaces, and finally indexed by frequency (so low integers correspond to frequent words). 
#### The integers 0, 1, and 2 are special: They represent the padding token, the start-of-sequence (SSS) token, and unknown words, respectively.

#### Lets decode them using the word index

In [9]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}

1641221/1641221 [==============================] - 0s 0us/step


In [10]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

In [11]:
"".join([id_to_word[id_] for id in X_train[0][:10]])

'<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>'

In [18]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised = True, with_info = True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\unnikris\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteZC1L1H\imdb_reviews-tr…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\unnikris\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteZC1L1H\imdb_reviews-te…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\unnikris\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteZC1L1H\imdb_reviews-un…

Dataset imdb_reviews downloaded and prepared to C:\Users\unnikris\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [19]:
for X_batch, y_batch in datasets["train"].batch(3).take(2):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative

Review: Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0 = Negative

Review: This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1 = Positive

Review: As others have mentioned, all the women that go nude in 

In [20]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value = b"<pad>"), y_batch

In [21]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(3, 59), dtype=string, numpy=
 array([[b'This', b'is', b'the', b'kind', b'of', b'film', b'for', b'a',
         b'snowy', b'Sunday', b'afternoon', b'when', b'the', b'rest',
         b'of', b'the', b'world', b'can', b'go', b'ahead', b'with',
         b'its', b'own', b'business', b'as', b'you', b'descend', b'into',
         b'a', b'big', b'arm', b'chair', b'and', b'mellow', b'for', b'a',
         b'couple', b'of', b'hours', b'Wonderful', b'performances',
         b'from', b'Cher', b'and', b'Nicolas', b'Cage', b'as', b'always',
         b'gently', b'row', b'the', b'plot', b'along', b'There', b'are',
         b'no', b'rapids', b'to', b'cr'],
        [b'As', b'others', b'have', b'mentioned', b'all', b'the',
         b'women', b'that', b'go', b'nude', b'in', b'this', b'film',
         b'are', b'mostly', b'absolutely', b'gorgeous', b'The', b'plot',
         b'very', b'ably', b'shows', b'the', b'hypocrisy', b'of', b'the',
         b'female', b'libido', b'When', b'men', b'are'

In [22]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [23]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [24]:
vocab_size = 10000
truncated_vocabulary = [ word for word, count in vocabulary.most_common()[:vocab_size]]

In [25]:
words = tf. constant(truncated_vocabulary)
words_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, words_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [16]:
table.lookup(tf.constant([b"This movie was faaaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10770]], dtype=int64)>

In [26]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [18]:
embed_size = 128
model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, mask_zero=True, input_shape=[None]), 
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.GRU(128),
        keras.layers.Dense(1, activation="sigmoid")])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=20)

Epoch 1/20
782/782 [==============================] - 35s 30ms/step - loss: 0.5347 - accuracy: 0.7239
Epoch 2/20
782/782 [==============================] - 22s 29ms/step - loss: 0.3500 - accuracy: 0.8540
Epoch 3/20
782/782 [==============================] - 24s 30ms/step - loss: 0.2011 - accuracy: 0.9303
Epoch 4/20
782/782 [==============================] - 24s 30ms/step - loss: 0.1451 - accuracy: 0.9475
Epoch 5/20
782/782 [==============================] - 27s 34ms/step - loss: 0.1086 - accuracy: 0.9610
Epoch 6/20
782/782 [==============================] - 24s 31ms/step - loss: 0.0807 - accuracy: 0.9722
Epoch 7/20
782/782 [==============================] - 21s 27ms/step - loss: 0.0733 - accuracy: 0.9739
Epoch 8/20
782/782 [==============================] - 28s 36ms/step - loss: 0.0531 - accuracy: 0.9812
Epoch 9/20
782/782 [==============================] - 28s 36ms/step - loss: 0.0369 - accuracy: 0.9873
Epoch 10/20
782/782 [==============================] - 28s 36ms/step - loss: 0.026

In [27]:
import tensorflow_hub as hub

In [28]:
model = keras.Sequential([
        hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", dtype = tf.string, input_shape = [], output_shape=[50]),
        keras.layers.Dense(128, activation = "relu"),
        keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [32]:
tf.random.set_seed(42)
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [34]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

In [36]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs = 5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5477 - accuracy: 0.7266
Epoch 2/5
782/782 [==============================] - 3s 4ms/step - loss: 0.5151 - accuracy: 0.7476
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5101 - accuracy: 0.7503
Epoch 4/5
782/782 [==============================] - 3s 4ms/step - loss: 0.5067 - accuracy: 0.7530
Epoch 5/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5039 - accuracy: 0.7551
